# Binary Black Hole

This notebook will get you started on your path to visualizing gravitational waves. We will load in data from a simulation of a binary black hole in-spiral.

In [ ]:
filename = 'data/qc0-mclachlan-cell.s5'

First let's load our libraries.

In [ ]:
import yt
import pysimulationio

## Finding the right configuration

It's good to know what configuraitons are available to read in. There are two ways, essentially, to access that information. The first is to use the `pysimulationio` library like so:

```python
project,file_handle =\
    pysimulationio.readProject(filename)
configuration_names =\
    [k for k in\
     project.configurations.iterkeys()]
file_handle.close()
```

The other way is to use prior knowledge of the simulation to generate the list of configurations. That's the tactic we'll use here.

In [ ]:
cname_template = 'iteration.{:0>10}-timelevel.0'
configuration_names = [cname_template.format(i)\
                       for i in range(0,46656+192,192)]

You can print the configuration names if you want. Fair warning: the list is long.

In [ ]:
configuration_names

## Loading in the Data

Now let's load in the data. The data is cell-centred, so it's easy. (Warning, this may take a while.)

In [ ]:
configuration_name = configuration_names[23]
ds = yt.load(filename,
            configuration = configuration_name)

## Gravitational Waves

Gravitational waves, as we see them on Earth, are measured in "strain" $h$, which has two polarizations: $h_{+}$ and $h_{x}$. The strain represents the fractional distortion of distance divided by total distance:

$$\frac{\Delta x}{x}$$

$h_{+}$ and $h_{x}$ (and indeed gravitational waves in general) are only defined far from the merging compact objects, in the so-called "wave zone." (Think electromagnetic waves. It's analogous.)

One way of extracting the gravitational wave information is to look at what information propagates along null directions (i.e., what travels at the speed of light). Four scalars are usually defined this way, called the "Weyl Scalars." Of particular interest to us is

$$\Psi_4 = C_{\alpha\beta\gamma\delta} n^\alpha \bar{m}^\beta n^\gamma \bar{m}^\delta$$

where $C_{\alpha\beta\gamma\delta}$ is the Weyl tensor and the set $\{l^\alpha,n^\alpha,m^\alpha,\bar{m}^\beta\}$ is a complex, null tetrad (collection of orthonormal basis vectors at each point).

For asymptotically flat spacetimes, $\Psi_4$ represents outgoing gravitational radiation. Indeed, if you go far enough away, it asymptotes to a particular combination of the derivatives of $h_+$ and $h_x$:

$$\Psi_4 \to -\ddot{h}_+ + i \ddot{h}_x$$

as $r\to\infty$.

If you output $\Psi_4$ in your simulation, you can plot in yt. In a parameter file, the settings look like:

```
IOHDF5::out_vars               = "
        WEYLSCAL4::Psi4r
        WEYLSCAL4::Psi4i
"
```

to generate 3D data. You can also generate the data on a 2D extraction surface, but we won't cover that here.

yt knows about the following fields related to gravitational waves:

In [ ]:
gw_fields = filter(lambda x: 'weyl' in x,
                   map(lambda y: y[1],
                       ds.derived_field_list))
for field in gw_fields:
    print field

The real part gives you the second derivative of $h_+$, the imaginary part gives you the second derivative of $h_x$.

The magnitude gives, very roughly, how strong the gravitational waves are.

The average energy density (it's only meaningful if you average over at least one wave period!) of gravitational waves is something like

$$\left\langle t^{00}\right\rangle = \frac{1}{4\pi}\left\langle \dot{h}_+^2 + \dot{h}_x^2\right\rangle$$

So the power is

$$\frac{d}{dt}\left\langle t^{00}\right\rangle = \frac{1}{2\pi} \left\langle \dot{h}_+\ddot{h}_+ + \dot{h}_x\ddot{h}_x\right\rangle$$

whereas 

$$|\Psi_4| = \sqrt{\ddot{h}_+^2 + \ddot{h}_x^2}$$

## Exercises

Plot various quantities related to gravitational waves. Unfortunately, analysing a whole simulation at once is not yet supported, so we recomend you stick to plotting $\Psi_4$ and friends. Here are some possible tasks:

* Volume render $Re(\Psi_4)$ or $Im(\Psi_4)$ and try and understand the structure. Does the shape remind you of anything?

* Make a slice or projection plot of $|\Psi_4|$

* Make a movie. (See below)

## Making a movie

This dataset has a lot of data in it, so it's actually possible to make a movie of the gravitational waves (or of the motion of the black holes). There's no "one-click" way to do this, but there are a few possible strategies. 

### Strategy 1:  save a bunch of files and stitch them together later

This is the easiet way to do things. Simply load several datasets in sequence and, for each one, save a plot. Then convert them into a gif using, say, `imagemagick`. Here's an example for just two or three frames:

In [ ]:
for j,i in enumerate(range(0,30,10)):
    # in this loop i = 0,10,20 and j = 0,1,2
    ds = yt.load(filename,
                 configuration = configuration_names[i])
    slc = yt.SlicePlot(ds,'z','lapse',width=10)
    slc.set_log('lapse',False)
    slc.save('qc0-frame-{}.png'.format(j))

Now we can use some bash code to generate the gif. (The `%%bash` at the top of the cell means the cell gets evaluated as bash code.)

In [ ]:
%%bash
convert -delay 100 qc0-frame-*.png qc0-movie.gif

If you ran the above code, you can view the movie (in gif format) by embedding it in the notebook with some html code:

In [ ]:
%%html 
<img src="qc0-movie.gif">

### Strategy 2: Use Python's functionality

Python contains a tool for making animations, the `matplotlib.animation` library. Therefore, you can also use this toolbox. The yt-project documentation has a tutorial on how to do this [here](http://yt-project.org/doc/cookbook/embedded_webm_animation.html). And [here's](https://jakevdp.github.io/blog/2012/08/18/matplotlib-animation-tutorial/) a nice tutorial that explains what's going on with the animation library. 

To embed in an IPython notebook, I prefer to use the awesome [jsanimation](https://github.com/jakevdp/JSAnimation) library. Here's an example.

In [ ]:
# load the appropriate libraries
from matplotlib import animation
from JSAnimation import IPython_display

# Set up the initial frame
ds = yt.load(filename,
                 configuration = configuration_names[0])
slc = yt.SlicePlot(ds,'z','lapse',width=10)
slc.set_log('lapse',False)
fig = slc.plots['lapse'].figure

# This function gets called to generate frame i
def animate(i):
    ds = yt.load(filename,
                 configuration = configuration_names[10*i])
    slc._switch_ds(ds) # feed a new dataset to the plot

# call the python animation library
# which uses the animate function to generate frames
# thanks to JSAnimation, it'll display inline
animation.FuncAnimation(fig,animate,frames=3,
                        interval=200,blit=False)

If you want to save the animation to a separate file, you can replace the last line with

```python
anim = animation.FuncAnimation(fig,animate,frames=3,
                               interval=200,blit=False)
anim.save('my-movie.mp4')
```